# 10.1 泛型資料型別

Ref: [泛型資料型別](https://rust-lang.tw/book-tw/ch10-01-syntax.html)

### 在函式中定義

> 當要使用泛型定義函數時，我們通常會將泛型置於函式簽名中指定參數與回傳值資料型別的位置。這樣做能讓我們的程式碼更具彈性並向呼叫者提供更多功能，同時還能防止重複程式碼。
>
> 接續我們 largest 函式的例子， __範例 10-4__  展示了兩個都在切片上尋找最大值的函式。我們要使用泛型將它們融合成一個函式。



In [2]:
// 範例 10-4：兩個名稱與其簽名中的型別都不同的函式

fn largest_i32(list: &[i32]) -> &i32 {
    let mut largest = &list[0];

    for item in list {
        if item > largest {
            largest = item;
        }
    }

    largest
}

fn largest_char(list: &[char]) -> &char {
    let mut largest = &list[0];

    for item in list {
        if item > largest {
            largest = item;
        }
    }

    largest
}

fn main() {
    let number_list = vec![34, 50, 25, 100, 65];

    let result = largest_i32(&number_list);
    println!("最大數字為 {}", result);
    assert_eq!(*result, 100);

    let char_list = vec!['y', 'm', 'a', 'q'];

    let result = largest_char(&char_list);
    println!("最大字元為 {}", result);
    assert_eq!(*result, 'y');
}
main()

最大數字為 100
最大字元為 y


()

上列兩個分別處理 `char` 以及 `i32` 的 `largest_char` 和 `largest_i32` 函式，我們可以將它們融合成一個函式，如 __範例 10-5__ 所示。

```rust
fn largest<T>(list: &[T]) -> &T {
```

## 10.1.1 定義共同行為

In [3]:
// 範例 10-5：使用泛型型別參數的 largest 函式，但現在還不能編譯

fn largest<T>(list: &[T]) -> &T {
    let mut largest = &list[0];

    for item in list {
        if item > largest {
            largest = item;
        }
    }

    largest
}

Error: binary operation `>` cannot be applied to type `&T`

<div>
    <img src="./does_not_compile.svg" width="100"/>
</div>

在這邊我們遇到的問題是，編譯器認為這邊可能會遇到不能直接做比較操作的型別，而建議我們把 `largest` 接受的型別定義好，這個部分會在 10.2 提及。

> 提示文字中提到了 `std::cmp::PartialOrd` 這個特徵（`trait`）。我們會在下個段落來討論特徵。
> 現在只需要知道 `largest` 本體無法適用於**所有可能**的 `T` 型別，因為我們想要在本體中比較型別 `T` 的數值，我們**只能在能夠排序的型別**中做比較。
> 要能夠比較的話，標準函式庫有提供 `std::cmp::PartialOrd` 特徵讓你可以針對你的型別來實作（請查閱附錄 C 來瞭解更多此特徵的細節）。照著提示文字的建議，我們限制 T 只對有實作 `PartialOrd` 的型別有效。這樣此範例就能編譯，因為標準函式庫有對 `i32` 與 `char` 實作 `PartialOrd`。

In [4]:
fn main() {
    let number_list = vec![34, 50, 25, 100, 65];

    let result = largest(&number_list);
    println!("最大數字為 {}", result);

    let char_list = vec!['y', 'm', 'a', 'q'];

    let result = largest(&char_list);
    println!("最大字元為 {}", result);
}

main()

Error: cannot find function `largest` in this scope

Error: cannot find function `largest` in this scope

---

## 10.1.2 在結構體中使用泛型

- 在下方我們就使用了泛型來定義一個結構體，這個結構體可以儲存任何型別的值，而這裡以 `i32` 為例。

In [5]:
// 範例 10-6：Point<T> 結構體的 x 與 y 會有型別 T 的數值

struct Point<T> {
    x: T,
    y: T,
}

fn main() {
    let integer = Point { x: 5, y: 10 };
    let float = Point { x: 1.0, y: 4.0 };
}

main()

()

- 輸入的時候必須是相同的型別，不然會出錯。下方編譯器在得知第一個參數為 `i32` 後，就會推斷出第二個參數也必須是 `i32`。

> 在此例中，當我們賦值 `5` 給 `x` 時，我們讓編譯器知道 `Point<T>` 實例中的泛型型別 `T` 會是**整數**。然後我們將 `4.0` 賦值給 `y`，這應該要和 `x` 有相同型別，所以我們會獲得以下錯誤：

In [6]:
// 範例 10-7：欄位 x 與 y 必須是相同型別，因為它們擁有相同的泛型資料型別 T

struct Point<T> {
    x: T,
    y: T,
}

fn main() {
    let wont_work = Point { x: 5, y: 4.0 };
}

main()

Error: mismatched types

- 如果有使用不一樣的型別，就要分開定義

In [7]:
// 範例 10-8：Point<T, U> 擁有兩個泛型型別，所以 x 和 y 可以有不同的型別數值

struct Point<T, U> {
    x: T,
    y: U,
}

fn main() {
    let both_integer = Point { x: 5, y: 10 };
    let both_float = Point { x: 1.0, y: 4.0 };
    let integer_and_float = Point { x: 5, y: 4.0 };
}
main()

()

---

## 10.1.3 在枚舉(`Result`, `Option`)中使用泛型

In [85]:
enum Option<T> {
    Some(T),
    None,
}

In [86]:
enum Result<T, E> {
    Ok(T),
    Err(E),
}

---

### 10.1.4 在方法（Method）中使用泛型

> 我們可以對結構體或枚舉定義方法（如第五章所述）並也可以使用泛型型別來定義。 __範例 10-9__ 展示了我們在 __範例 10-6__ 定義的結構體 `Point<T>` 並實作了一個叫做 `x` 的方法。

In [8]:
// 範例 10-9：在 Point<T> 結構體實作一個方法叫做 x，其會回傳 x 欄位中型別為 T 的參考

struct Point<T> {
    x: T,
    y: T,
}

impl<T> Point<T> {
    fn x(&self) -> &T {
        &self.x
    }
}

fn main() {
    let p = Point { x: 5, y: 10 };

    println!("p.x = {}", p.x());
}
main()

p.x = 5


()

> 注意到我們需要在 `impl` 宣告 `T`，才有 `T` 可以用來標明我們在替型別 `Point<T>` 實作其方法。在 `impl` 之後宣告泛型型別 `T`，Rust 可以識別出 `Point` 尖括號內的型別為泛型型別而非實際型別。我們其實可以選用不同的泛型參數名稱，而不用和結構體定義的泛型參數一樣，不過通常使用相同名稱還是比較常見。無論該泛型型別最終會是何種實際型別，任何方法在有宣告泛型型別的 `impl` 內，都會被定義成適用於各種型別實例。

### a. 限制方法所使用的泛型型別

> 當我們在定義方法時，我們也可以對泛型型別加上些限制。舉例來說，我們可以只針對 `Point<f32>` 的實例來實作方法，而非適用於任何泛型型別的 `Point<T>` 實例。

而這個時候變不用加`T`

In [10]:
// 範例 10-10：一個只適用於擁有泛型 T 結構體其中的特定實際型別的 impl 區塊

struct Point<T> {
    x: T,
    y: T,
}

impl<T> Point<T> {
    fn x(&self) -> &T {
        &self.x
    }
}

impl Point<f32> {
    fn distance_from_origin(&self) -> f32 {
        (self.x.powi(2) + self.y.powi(2)).sqrt()
    }
}

fn main() {
    let p = Point { x: 5, y: 10 };

    println!("p.x = {}", p.x());
}
main()

p.x = 5


()

實作方法也可以不用到泛型型別，如下所示。

In [15]:
struct Point02<T> {
    x: T,
    y: T,
}

impl<T> Point02<T> {
    fn the_utlmatic_answer_of_all(&self) -> i32 {
        42
    }
}

fn main() {
    let p = Point02 { x: 5, y: 10 };

    println!("p.the_utlmatic_answer_of_all = {}", p.the_utlmatic_answer_of_all());
}
main()

p.the_utlmatic_answer_of_all = 42


()

### b. 多個泛型的例子

In [3]:
// 範例 10-11：結構體定義中使用不同的泛型型別的方法

struct PointMulti<X1, Y1> {
    x: X1,
    y: Y1,
}

impl<X1, Y1> PointMulti<X1, Y1> {
    fn mixup<X2, Y2>(self, other: PointMulti<X2, Y2>) -> PointMulti<X1, Y2> {
        PointMulti {
            x: self.x,
            y: other.y,
        }
    }
}

fn main() {
    let p1 = PointMulti { x: 5, y: 10.4 };
    let p2 = PointMulti { x: "Hello", y: 'c' };

    let p3 = p1.mixup(p2);

    println!("p3.x = {}, p3.y = {}", p3.x, p3.y);
}
main()

p3.x = 5, p3.y = c


()

- 實作（`impl`）中使用的泛型型別註解名稱得必須和實作對象的一致，否則編譯器會無法辨認

In [4]:
impl<T, U> PointMulti<X2, Y2> {

}

Error: cannot find type `X2` in this scope

Error: cannot find type `Y2` in this scope

## 10.1.5 使用泛型的程式碼效能

> 你可能會好奇當你使用泛型型別參數會不會有執行時的消耗。**好消息是使用泛型型別不會比使用實際型別還來的慢**。
> 
> Rust 在編譯時對使用泛型的程式碼進行單態化（monomorphization）。單態化是個讓泛型程式碼轉換成特定程式碼的過程，在編譯時填入實際的型別。在此過程中，編譯器會做與我們在範例 10-5 建立泛型函式相反的事：編譯器檢查所有泛型程式碼被呼叫的地方，並依據泛型程式碼被呼叫的情況產生實際型別的程式碼。

你~~煮出來的義麵碼~~寫出來的程式碼：

```rust
let integer = Some(5);
let float = Some(5.0);
```
> 當 Rust 編譯此程式碼時中，他會進行單態化。在此過程中，會讀取 Option<T> 實例中使用的數值並識別出兩種 Option<T>：一種是 i32 而另一種是 f64。接著它就會將 Option<T> 的泛型定義展開為兩種定義 i32 與 f64，以此替換函式定義為特定型別。


編譯器執行的：

```rust
enum Option_i32 {
    Some(i32),
    None,
}

enum Option_f64 {
    Some(f64),
    None,
}

fn main() {
    let integer = Option_i32::Some(5);
    let float = Option_f64::Some(5.0);
}
```

編譯器實際上會去確認泛型中出現的型別，然後分別把他們真正實作後，再去執行